In [35]:
%matplotlib notebook
import json
import numpy as np
import os
import pandas as pd
import sys
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import time, datetime, math

DATA_DIR = 'data/twitter/predict_ii/'

def print_time(unix, str=''):
    print(str, time.ctime(int(unix)))
    
def get_readable_time(unix):
    return [time.ctime(int(u)) for u in unix]

In [21]:
twitter_df = pd.DataFrame()
for filename in os.listdir(DATA_DIR):
    content = pd.read_csv(DATA_DIR + filename, sep='\t')
    twitter_df = pd.concat([twitter_df, content])

print(twitter_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18995 entries, 0 to 10
Data columns (total 13 columns):
currency      12370 non-null object
tweet         18995 non-null object
username      18995 non-null object
favorites     18995 non-null int64
retweets      18995 non-null int64
date          18995 non-null int64
id            18995 non-null int64
permalink     18995 non-null object
geo           0 non-null float64
hashtags      6810 non-null object
mentions      4939 non-null object
sent          18995 non-null float64
vader_sent    18995 non-null float64
dtypes: float64(3), int64(4), object(6)
memory usage: 2.0+ MB
None


In [24]:
twitter_df.to_pickle('data/twitter/milica_combined.pkl')

In [70]:
twitter_df = twitter_df.sort_values(by='date', ascending=True)
twitter_df.corr()

,favorites,retweets,date,id,geo,sent,vader_sent
favorites,1.000000,0.886469,0.016622,0.016622,NaN,-0.000591,-0.003964
retweets,0.886469,1.000000,0.007405,0.007405,NaN,-0.003494,-0.010960
date,0.016622,0.007405,1.000000,1.000000,NaN,0.003803,-0.000831
id,0.016622,0.007405,1.000000,1.000000,NaN,0.003803,-0.000831
geo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sent,-0.000591,-0.003494,0.003803,0.003803,NaN,1.000000,0.244321
vader_sent,-0.003964,-0.010960,-0.000831,-0.000831,NaN,0.244321,1.000000


In [69]:
dates = get_readable_time(list(twitter_df['date']))
#dates

time_diff = twitter_df['date'].diff().sort_values(ascending=False)
time_diff.describe()

count    18994.000000
mean       541.249395
std       1078.725470
min          0.000000
25%        109.000000
50%        288.000000
75%        668.000000
max      86950.000000
Name: date, dtype: float64

In [64]:
time_diff = twitter_df['date'].diff().sort_values().reset_index().query('date<1500')
time_diff
#time_diff.describe()

,index,date
0,30,0.0
1,133,0.0
2,124,0.0
3,98,0.0
4,97,0.0
5,176,0.0
6,174,0.0
7,205,0.0
8,87,0.0
9,139,0.0
